In [ ]:
from tkinter import Tk, Button
from tkfontchooser import Font
from copy import deepcopy
class GUI:
  def __init__(self):
    self.app = Tk()
    self.app.title('TicTacToe')
    self.app.resizable(width=True, height=True)
    self.board = Board()
    self.font = Font(family="Courier", size=32)
    self.buttons = {}

    for x,y in self.board.fields:
      putvalues = lambda x=x,y=y: self.move(x,y)
      button = Button(self.app, command=putvalues, font=self.font, width=10, height=3)
      button.grid(column=x,row = y)
      self.buttons[x,y] = button

    self.update()


  def move(self,x,y):
    self.app.config(cursor="watch")
    self.app.update()
    self.board = self.board.move(x,y)
    self.update()
    #minimax se jaane ka rasta
    move = self.board.best()
    if move:
      self.board = self.board.move(*move)
      self.update()
    self.app.config(cursor="")

  def update(self):
    for (x,y) in self.board.fields:
      text = self.board.fields[x,y]
      self.buttons[x,y]['text'] = text
      self.buttons[x,y]['disabledforeground'] = 'blue'
      if text==self.board.empty:
        self.buttons[x,y]['state'] = 'normal'
      else:
        self.buttons[x,y]['state'] = 'disabled'
    winning = self.board.won()
    if winning:
      for x,y in winning:
        self.buttons[x,y]['disabledforeground'] = 'red'
      for x,y in self.buttons:
        self.buttons[x,y]['state'] = 'disabled'
    for (x,y) in self.board.fields:
      self.buttons[x,y].update()

  def mainloop(self):
    self.app.mainloop()


class Board:
  def __init__(self,other=None):
    self.user = 'X'
    self.bot = 'O'
    self.empty = '__'
    self.size = 3
    self.fields = {}
    for y in range(self.size):
      for x in range(self.size):
        self.fields[x,y] = self.empty
    # copy constructor
    if other:
      self.__dict__ = deepcopy(other.__dict__)
  def move(self,x,y):
    board = Board(self)
    board.fields[x,y] = board.user
    (board.user,board.bot) = (board.bot,board.user)
    return board

  def tied(self):
    for (x,y) in self.fields:
      if self.fields[x,y]==self.empty:
        return False
    return True
  def won(self):
    # horizontal
    for y in range(self.size):
      winning = []
      for x in range(self.size):
        if self.fields[x,y] == self.bot:
          winning.append((x,y))
      if len(winning) == self.size:
        return winning
    # vertical
    for x in range(self.size):
      winning = []
      for y in range(self.size):
        if self.fields[x,y] == self.bot:
          winning.append((x,y))
      if len(winning) == self.size:
        return winning
    # diagonal
    winning = []
    for y in range(self.size):
      x = y
      if self.fields[x,y] == self.bot:
        winning.append((x,y))
    if len(winning) == self.size:
      return winning
    # other diagonal(second diagonal)
    winning = []
    for y in range(self.size):
      x = self.size-1-y
      if self.fields[x,y] == self.bot:
        winning.append((x,y))
    if len(winning) == self.size:
      return winning
    # default
    return None

  def __minimax(self, user):
    if self.won():
      if user:
        return (-1,None)
      else:
        return (+1,None)
    elif self.tied():
      return (0,None)
    elif user:
      best = (-2,None)
      for x,y in self.fields:
        if self.fields[x,y]==self.empty:
          value = self.move(x,y).__minimax(not user)[0]
          if value>best[0]:
            best = (value,(x,y))
      return best
    else:
      best = (+2,None)
      for x,y in self.fields:
        if self.fields[x,y]==self.empty:
          value = self.move(x,y).__minimax(not user)[0]
          if value<best[0]:
            best = (value,(x,y))
      return best

  def best(self):
    return self.__minimax(True)[1]

if __name__ == '__main__':
  GUI().mainloop()
